### Below three cells are working references

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.request
url = "http://mlg.ucd.ie/modules/yalp/automotive_list.html"
response = urllib.request.urlopen(url)
text = response.read().decode()
# print(text)
lines = text.strip().split("\n")
for l in lines:
     print(l)


In [ ]:
URL = 'http://mlg.ucd.ie/modules/yalp/automotive_list.html'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

In [ ]:
# Open the url and parse the data in the html page

import bs4
import urllib.request
url = "http://mlg.ucd.ie/modules/yalp/automotive_list.html"
response = urllib.request.urlopen(url)
html = response.read().decode()
parser = bs4.BeautifulSoup(html,"html.parser")
print(parser.prettify())


### Code begins from here

***Task 1: Select three review categories of your choice. Scrape all reviews for each
category and store them as three separate datasets. For each review, you
should store the review text and a class label (i.e. whether the review is
“positive” or “negative”).***

In [21]:
# import all the libraries required for the assignment
import bs4
import urllib.request
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score, RepeatedKFold

In [16]:
#Define function to open the 3 different category url, read their content and store in separate dataframes


def get_url(url):
    response = urllib.request.urlopen(url)
    html = response.read().decode()
#     parser = bs4.BeautifulSoup(html,"html.parser")
    return html

automotive_url = "http://mlg.ucd.ie/modules/yalp/automotive_list.html"
hairsalon_url ="http://mlg.ucd.ie/modules/yalp/hair_salons_list.html"
cafes_url= "http://mlg.ucd.ie/modules/yalp/cafes_list.html"

soup= get_url(cafes_url)
parser = bs4.BeautifulSoup(soup,"html.parser")
# print(parser.prettify())
# print(soup)

In [17]:
# find 'a' tags to get the links of the businesses

temp_business_urls =[]
base_url = "http://mlg.ucd.ie/modules/yalp/"
business_list = parser.find(id= 'all')
# print(business_list)

def get_business_urls():
    for match in business_list.find_all('a'):
        link = match.get('href')
        temp_business_urls.append(base_url+link)
    return temp_business_urls

business_urls = get_business_urls()
# print(len(business_urls))

# for bus in get_business_urls():
#     print(bus)


In [18]:
# Define a function to open the html pages above and read content

review_data = []
def open_html():
    headers = []
    headers.append("title")
    headers.append("review_text")
    headers.append("rating")
    review_data.append(headers)
    for b_url in business_urls:
        response = urllib.request.urlopen(b_url)
        html = response.read().decode()
        soup = bs4.BeautifulSoup(html,"html.parser")
#         print(soup)
        title = soup.find('h3',{'class': 'info'}).get_text()
#         print(title)
        reviews = soup.find(id ='all')
        
        for review in reviews.find_all(class_='review'):
            content =[]
            review_text = review.find ('p',{'class':'review-text'}).get_text()
            # print(review_text)
            rating_text = review.find('img')['alt']
            if rating_text == '4-star':
                rating = "positive"
            elif rating_text == '5-star':
                rating = "positive"
            elif rating_text == '1-star':
                rating = "negative"
            elif rating_text == '2-star':
                rating = "negative"
            elif rating_text == '3-star':
                rating = "negative"
            else:
                #for any other rating the value is assumed to be negative
                rating = "negative"
#             print(rating)
            content.append(title)
            content.append(review_text)
            content.append(rating)  
            review_data.append(content)
    return


In [19]:
#Call the function to read review data
open_html()
#print(review_data)

In [10]:

column_names = review_data.pop(0)
df_automotive = pd.DataFrame(review_data,columns=column_names)
df_automotive.head(10)


,title,review_text,rating
0,Yalp — 7-Eleven,The man that was working tonight (8-12-17) was...,negative
1,Yalp — 7-Eleven,Chris is a very rude person. Gave me an attitu...,negative
2,Yalp — 7-Eleven,One of my favorite gas station to stop at. The...,positive
3,Yalp — 7-Eleven,Oh thank Heaven for Seven Eleven! I don't know...,negative
4,Yalp — 7-Eleven,Five stars because of the guy who works weekda...,positive
5,Yalp — AA Auto Care,I had the best experience here!!!!!! All the e...,positive
6,Yalp — AA Auto Care,My objective with this shop was to find a good...,negative
7,Yalp — AA Auto Care,Absolute worst service ever. Went in for oil c...,negative
8,Yalp — AA Auto Care,Love love love their customer service. I come ...,positive
9,Yalp — AA Auto Care,I took my friend to this shop to get her oil c...,negative


In [15]:
column_names = review_data.pop(0)
df_hairsalon = pd.DataFrame(review_data,columns=column_names)
df_hairsalon.head(10)

,title,review_text,rating
0,Yalp — Ab's Barber Shop,"One of the best barbershops I've been to, with...",positive
1,Yalp — Ab's Barber Shop,Took my son in for a haircut. Barber was great...,positive
2,Yalp — Ab's Barber Shop,"Walked in, said hi. The only barber in there d...",negative
3,Yalp — Ab's Barber Shop,I came here 10 minutes before 9am to get a hai...,negative
4,Yalp — Ab's Barber Shop,"Great haircut. No fuss no muss, I asked for la...",positive
5,Yalp — Adore Salon and Boutique,Love this place great atmosphere and energy. V...,positive
6,Yalp — Adore Salon and Boutique,Loved this salon. It is very clean an smells g...,positive
7,Yalp — Adore Salon and Boutique,I will never recommend this business to anyone...,negative
8,Yalp — Adore Salon and Boutique,I bought a Groupon from Sandra Lynn Rosendin f...,positive
9,Yalp — Adore Salon and Boutique,Ester is fabulous with hair. She colors and no...,positive


In [20]:
column_names = review_data.pop(0)
df_cafes = pd.DataFrame(review_data,columns=column_names)
df_cafes.head(10)

,title,review_text,rating
0,Yalp — ABC Bakery Coffee Shop,Pros: Lots of items you would not expect from ...,positive
1,Yalp — ABC Bakery Coffee Shop,Best egg-tarts in town! There's really not muc...,positive
2,Yalp — ABC Bakery Coffee Shop,I've been to ABC Bakery a few times since I re...,negative
3,Yalp — ABC Bakery Coffee Shop,"FYI, Closed Monday's New ownership for about 1...",negative
4,Yalp — ABC Bakery Coffee Shop,The inside may not look like much but they mak...,positive
5,Yalp — ABC Bakery Coffee Shop,I usually come for the egg tart and it is one ...,positive
6,Yalp — ABC Bakery Coffee Shop,"Everything, I meant everything were good. Were...",positive
7,Yalp — ABC Bakery Coffee Shop,"It's more like 3 and a 1/2 stars, but if value...",negative
8,Yalp — ABC Bakery Coffee Shop,This place got to have the best Hong Kong styl...,positive
9,Yalp — ABC Bakery Coffee Shop,Make a quick stop here while waiting for dim s...,positive


# Task 1 Conclusion: 
***Three dataframes created :***
***- df_automotive***
***- df_hairsalon***
***-df_cafes***

# Task 2:For each of the three category datasets:

***a. From the reviews in this category, apply appropriate preprocessing steps to
create a numeric representation of the data, suitable for classification.***

***b. Build a classification model to distinguish between “positive” and “negative”
reviews using one of the following classifiers:
Naive Bayes, Logistic Regression, Random Forests***

***c. Test the predictions of the classification model using an appropriate
evaluation strategy. Report and discuss the evaluation results in your
notebook.***

### Data Pre-processing

In [22]:
# document and target creation for automotive dataset
documents_auto = df_automotive["review_text"]
target_auto = df_automotive["rating"]

In [23]:
# document and target creation for hairsalon dataset
documents_hairsalon = df_hairsalon["review_text"]
target_hairsalon = df_hairsalon["rating"]

In [24]:
# document and target creation for cafes dataset
documents_cafes = df_cafes["review_text"]
target_cafes = df_cafes["rating"]

In [26]:
# check class bifurcation for each category
print("for automotive: " , target_auto.value_counts(),'\n',
      "for hairsalon: " ,target_hairsalon.value_counts(),'\n',
      "for cafes: ", target_cafes.value_counts()) 

for automotive:  positive    2544
negative    1618
rating         5
Name: rating, dtype: int64 
 for hairsalon:  positive    1558
negative     442
Name: rating, dtype: int64 
 for cafes:  positive    1462
negative     538
Name: rating, dtype: int64


In [28]:
# Produce vector representation of each of the documents

vectorizer = CountVectorizer( min_df = 10, stop_words=None )
auto = vectorizer.fit_transform(documents_auto)
hairsalon = vectorizer.fit_transform(documents_hairsalon)
cafes = vectorizer.fit_transform(documents_cafes)
print("auto:" , auto.shape)
print("hairsalon:" , hairsalon.shape)
print("cafes", cafes.shape)

auto: (4167, 2774)
hairsalon: (2000, 1510)
cafes (2000, 1727)


In [31]:
# check the distinct terms in each of the vectors

terms_auto = vectorizer.get_feature_names()
terms_hairsalon = vectorizer.get_feature_names()
terms_cafes= vectorizer.get_feature_names()
print("Vocabulary has %d distinct auto terms" % len(terms_auto),'\n',
     "Vocabulary has %d distinct hairsalon terms" % len(terms_hairsalon),'\n',
     "Vocabulary has %d distinct cafes terms" % len(terms_cafes))

# Display a set of sample terms
print(terms_auto[50:120])

Vocabulary has 1727 distinct auto terms 
 Vocabulary has 1727 distinct hairsalon terms 
 Vocabulary has 1727 distinct cafes terms
['although', 'always', 'am', 'amazing', 'ambiance', 'ambience', 'americano', 'amount', 'an', 'and', 'another', 'answer', 'any', 'anyone', 'anything', 'anyway', 'anywhere', 'apart', 'apparently', 'appetizer', 'appetizers', 'apple', 'appreciate', 'appreciated', 'are', 'area', 'areas', 'aren', 'around', 'arrived', 'art', 'arugula', 'as', 'asian', 'ask', 'asked', 'asking', 'assortment', 'at', 'ate', 'atmosphere', 'attention', 'attentive', 'attitude', 'au', 'authentic', 'available', 'avenue', 'average', 'avocado', 'avoid', 'away', 'awesome', 'awful', 'back', 'bacon', 'bad', 'bag', 'bagel', 'bagels', 'baked', 'bakery', 'balance', 'balanced', 'banana', 'bar', 'barely', 'barista', 'baristas', 'bars']


# Classification and Evaluation 

In [33]:
# building a k-NN model, and evaluating its performance using a single training/test split
# by taking 70:30 training and test set

# for automotive category:

data_train, data_test, target_train, target_test = train_test_split(auto, target_auto, test_size=0.3)

In [34]:
print("Training set has %d examples" % data_train.shape[0] )
print("Test set has %d examples" % data_test.shape[0] )

Training set has 2916 examples
Test set has 1251 examples


In [35]:
# Prepare the k-NN classification model, for 3 nearest neighbours in this case:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(data_train, target_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [36]:
#Make predictions for the reviews in the test set.
predicted = model.predict(data_test)
predicted

array(['negative', 'positive', 'positive', ..., 'negative', 'positive',
       'negative'], dtype=object)

In [37]:
print("Accuracy = %.2f" % accuracy_score(target_test, predicted) )

Accuracy = 0.75


In [40]:
cm = confusion_matrix(target_auto, predicted,labels=['negative','positive'])
print(cm)

ValueError: Found input variables with inconsistent numbers of samples: [4167, 1251]